# EXTRACCIÓN DE DATA

In [1]:
# librerías a utilizar
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta, date

In [ ]:
# funciones necesarias
# Creamos funcion para determinar si ya creamos csv de los datos de x día
def archivo_existe(nombre_archivo, carpeta):
    return (Path(carpeta)/nombre_archivo).exists()

# Creamos funcion que corrobora si ya tenemos todos los datos de cierto día:
def archivos_totales(diccionario, carpeta, fecha):
    try:
        # Cantidad de datos en cierto día registrados
        datos_dict = len(diccionario[fecha])
        contador = 0
        # Cantidad de datos en la carpeta (buses_outputs o metro_outputs) listados
        largo_carpeta = list(carpeta.glob('*csv'))
        for dato in largo_carpeta:
            if fecha in dato.stem:
                contador += 1
        # Corroborar si ambas cantidades son iguales
        return contador == datos_dict
    # Si no hemos revisado datos de esa fecha, o sea no existe su llave
    except KeyError:
        return False

# Creamos funcion que añada nuevos datos al dataframe de cierto día  
def anadir_datos(nuevos_datos, ruta):
    # Dataframe de los datos nuevos
    df_nuevos_datos = pd.concat(nuevos_datos, ignore_index=True)
    print(f'Dentro de anadir_datos')
    # Si ya tenemos datos en dicha fecha:
    if ruta.exists():
        # Abrimos los datos ya existentes
        datos_guardados = pd.read_csv(ruta)
        # Les añadimos los nuevos datos recolectados
        nuevo_csv = pd.concat([datos_guardados, df_nuevos_datos], ignore_index=True)
        print(f'Ahora nuevo csv')
        # volvemos a guardar en la ruta que estaban los datos ya existentes
        nuevo_csv.to_csv(ruta, index=False, encoding='utf-8')
    else:
        # si no teníamos datos en dicha fecha
        nuevo_csv = df_nuevos_datos
        print(f'No en ruta.exists')
        # agregamos los datos a la ruta 
        nuevo_csv.to_csv(ruta, index=False, encoding='utf-8')
    return nuevo_csv

In [ ]:
# Creamos las rutas a la carpeta de la micro:
carpeta_bus = Path('buses_outputs')
# Creamos la carpeta para guardar los datos:
csv_micro = Path('csv_micros')
csv_micro.mkdir(exist_ok=True)
# Determinamos formato de tiempo que tienen los distintos csv
tiempo_hora_minuto = '%d-%m-%Y-%H-%M'
# Determinamos fecha inicial y final
fecha_inicio = date(2025, 10, 12)
# Fecha en la que empezamos la extracción
fecha_actual = fecha_inicio
# Para los datos de la micro
info_df_diario = []
# Enlistamos los datos de 'buses_outputs'
recorrer = list(carpeta_bus.glob('*csv'))
# diccionario micros para ver si faltan agregar datos de un día
micro_dict = dict()
# Los recorremos para empezar a crear los DataFrame
for i in range(len(recorrer)): 
    # Ponemos en el formato de fecha de los datos extraídos: día, mes y año
    fecha_actual_formato = fecha_actual.strftime('%d-%m-%Y')
    # verificamos si ya tenemos un csv para los datos del archivo a realizar
    if archivo_existe(f'{fecha_actual_formato}_datos.csv', csv_micro) and archivos_totales(micro_dict, carpeta_bus, f'fecha_actual_formato'):
       # Si ya lo tenemos, ahora buscaremos los datos del siguiente día
        fecha_actual+= timedelta(days=1)
        continue
    # Nos quedamos con la fecha del archivo que estamos leyendo
    archivo_fecha = recorrer[i].stem[6:]
    try: 
        # Si el archivo de esta iteración es valor de la llave de la fecha actual
        if archivo_fecha in micro_dict[fecha_actual_formato]:
            continue
        else:
            # si no lo es, lo añadimos
            micro_dict[fecha_actual_formato].add(archivo_fecha)
    # Si esa llave no existía (no habíamos revisado nada de ese día), la creamos
    except KeyError:
        # La creamos en un set para hacer más fácil su búsqueda después
        micro_dict[fecha_actual_formato] = {f'{archivo_fecha}'}
    # Convertimos la fecha del archivo al formato tiempo_hora_minuto
    hora_minuto = datetime.strptime(archivo_fecha, tiempo_hora_minuto)
    # Leemos el csv actual
    dataframe_archivo = pd.read_csv(recorrer[i], encoding='utf-8')
    # Creamos una columna que incluya la hora y el minuto
    dataframe_archivo['Hour_minute'] = f'{hora_minuto.hour:02d}:{hora_minuto.minute:02d}'
    # añadimos el df creado a nuestra lista de información por día
    info_df_diario.append(dataframe_archivo)
    try:
        # Si la fecha actual no está en el siguiente elemento, se acabaron los datos del
        # día y debemos guardar todo lo recolectado
        if fecha_actual_formato not in recorrer[i+1].stem:
            # creamos la ruta donde guardaremos los datos
            ruta = csv_micro / f'{fecha_actual_formato}_datos.csv'
            # Usamos la función definida en el primer bloque, así se ve 
            # si se escribe recién el archivo o se le añaden datos
            anadir_datos(info_df_diario, ruta)
            # Vaciamos la info del día pues vamos a cambiarlo
            info_df_diario = []
            # Pasamos al día siguiente
            fecha_actual += timedelta(days=1)
    # Si llegamos al final de la lista no se podrá hacer i+1
    except IndexError as error:
        print(f"ocurrió un error {error}, o sea, llegamos al final de la lista")
        # creamos ruta para el último día de datos
        print(f'En el indexerror, trabajamos con ruta {ruta}')
        # la misma lógica que dentro del try
        ruta = csv_micro / f'{fecha_actual_formato}_datos.csv'
        anadir_datos(info_df_diario, ruta)
        # Vaciamos info del día
        info_df_diario = []

['datos_12-10-2025-19-01.csv',
 'datos_12-10-2025-20-00.csv',
 'datos_12-10-2025-22-00.csv',
 'datos_13-10-2025-06-00.csv',
 'datos_13-10-2025-08-00.csv',
 'datos_13-10-2025-10-00.csv',
 'datos_13-10-2025-12-00.csv',
 'datos_13-10-2025-14-00.csv',
 'datos_13-10-2025-16-00.csv',
 'datos_13-10-2025-18-00.csv',
 'datos_13-10-2025-20-00.csv',
 'datos_13-10-2025-22-00.csv',
 '.ipynb_checkpoints',
 'datos_14-10-2025-06-00.csv',
 'datos_14-10-2025-08-00.csv',
 'datos_14-10-2025-10-00.csv',
 'datos_14-10-2025-12-00.csv',
 'datos_14-10-2025-14-00.csv',
 'datos_14-10-2025-16-00.csv',
 'datos_14-10-2025-18-00.csv',
 'datos_14-10-2025-20-00.csv',
 'datos_14-10-2025-22-00.csv',
 'datos_15-10-2025-06-00.csv',
 'datos_15-10-2025-08-00.csv',
 'datos_15-10-2025-10-00.csv',
 'datos_15-10-2025-12-00.csv',
 'datos_15-10-2025-14-00.csv',
 'datos_15-10-2025-16-00.csv',
 'datos_15-10-2025-18-00.csv',
 'datos_15-10-2025-20-00.csv',
 'datos_15-10-2025-22-00.csv']

In [ ]:
# Ahora vamos a ver los datos del metro
carpeta_metro = Path('metro_outputs')
csv_metro = Path('csv_metro')
csv_metro.mkdir(exist_ok=True)
# Añadimos los datos diarios del metro
df_metro_diario = []
# Partimos con la fecha de inicio
fecha_actual_metro = fecha_inicio
datos = list(carpeta_metro.glob('*csv'))
# creamos diccionario para ver si faltan agregar los datos de un día
metro_dict = dict()
for i in range(len(datos)):
    # Trabajamos la fecha según como salen los datos de carpeta_metro
    formato_metro = fecha_actual_metro.strftime('%d-%m-%Y')
    if archivo_existe(f'{formato_metro}_datos.csv', carpeta_metro) and archivos_totales(metro_dict, carpeta_metro, f'{formato_metro}'):
        fecha_actual_metro += timedelta(days=1)
        continue
    fecha_a_revisar = datos[i].stem[6:]
    try: 
        if fecha_a_revisar in metro_dict[formato_metro]:
            continue
        else:
            metro_dict[formato_metro].add(fecha_a_revisar)
    except KeyError:
        metro_dict[formato_metro] = {f'{fecha_a_revisar}'}
    hora_minuto = datetime.strptime(fecha_a_revisar, tiempo_hora_minuto)
    dataframe_actual = pd.read_csv(datos[i], encoding='utf-8')
    dataframe_actual['Hora_minuto'] = f'{hora_minuto.hour:02d}:{hora_minuto.minute:02d}'
    df_metro_diario.append(dataframe_actual)
    try:
        # Si la fecha actual no está en el siguiente elemento, se acabaron los datos del
        # día y debemos guardar todo lo recolectado
        if formato_metro not in datos[i+1].stem:
            # creamos la ruta donde guardaremos los datos
            ruta = csv_metro / f'{fecha_actual_metro}_datos.csv'
            anadir_datos(df_metro_diario, ruta)
            df_metro_diario = []
            # Sumamos un día a la fecha actual
            fecha_actual_metro += timedelta(days=1)
    # Si llegamos al final de la lista no se podrá hacer i+1
    except IndexError as error:
        # creamos ruta para el último día de datos
        ruta = csv_metro / f'{fecha_actual_metro}_datos.csv'
        anadir_datos(df_metro_diario, ruta)
        df_metro_diario = []   